In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/tylerhouchin/Desktop/lens_inspection/deep-burner-360219-d5748f56761f.json"


In [2]:
#Creating a dataset
#This block creates a dataset wherein each image can only be assigned to one label (defect/normal) aka MULTICLASS

from google.cloud import automl

project_id = "deep-burner-360219"
display_name = "dataset1"

client = automl.AutoMlClient()

# A resource that represents Google Cloud Platform location.
project_location = f"projects/{project_id}/locations/us-central1"
# Specify the classification type
# Types:
# MultiLabel: Multiple labels are allowed for one example.
# MultiClass: At most one label is allowed per example.
# https://cloud.google.com/automl/docs/reference/rpc/google.cloud.automl.v1#classificationtype
metadata = automl.ImageClassificationDatasetMetadata(
    classification_type=automl.ClassificationType.MULTICLASS
)
dataset = automl.Dataset(
    display_name=display_name,
    image_classification_dataset_metadata=metadata,
)

# Create a dataset with the dataset metadata in the region.
response = client.create_dataset(parent=project_location, dataset=dataset, timeout=300)

created_dataset = response.result()

# Display the dataset information
print("Dataset name: {}".format(created_dataset.name))
print("Dataset id: {}".format(created_dataset.name.split("/")[-1]))

Dataset name: projects/897271433543/locations/us-central1/datasets/ICN4187945781954609152
Dataset id: ICN4187945781954609152


In [3]:
#Importing items into the dataset
#This block imports images into our created dataset by specifying filepath of previously created .csv 
#The csv must be stored in the same bucket as our images
#Format of csv is: one row per image; columns are use, file path, label

dataset_id = "ICN4187945781954609152"
path = "gs://deep-burner-360219-vcm/lenses.csv"

# Get the full path of the dataset.
dataset_full_id = client.dataset_path(project_id, "us-central1", dataset_id)
# Get the multiple Google Cloud Storage URIs
input_uris = path.split(",")
gcs_source = automl.GcsSource(input_uris=input_uris)
input_config = automl.InputConfig(gcs_source=gcs_source)
# Import data from the input URI
response = client.import_data(name=dataset_full_id, input_config=input_config)

print("Processing import...")
print("Data imported. {}".format(response.result()))


Processing import...
Data imported. 


Image labeleing is done using the [Google Console](https://cloud.google.com/vision/automl/docs/create-datasets#automl_vision_classification_create_dataset-python:~:text=items%20in%20the-,AutoML%20Vision%20UI,-%2C%20select%20the%20dataset)
.This is just verification as the lableing was provided in our lenses.csv

In [4]:
#This block begins to train our model

display_name = "inspection_model_v1"


# A resource that represents Google Cloud Platform location.
project_location = f"projects/{project_id}/locations/us-central1"
# Leave model unset to use the default base model provided by Google
# train_budget_milli_node_hours: The actual train_cost will be equal or
# less than this value.
# https://cloud.google.com/automl/docs/reference/rpc/google.cloud.automl.v1#imageclassificationmodelmetadata
metadata = automl.ImageClassificationModelMetadata(
    train_budget_milli_node_hours=24000
)
model = automl.Model(
    display_name=display_name,
    dataset_id=dataset_id,
    image_classification_model_metadata=metadata,
)

# Create a model with the model metadata in the region.
response = client.create_model(parent=project_location, model=model)

print("Training operation name: {}".format(response.operation.name))
print("Training started...")

Training operation name: projects/897271433543/locations/us-central1/operations/ICN3543064894955847680
Training started...
